# First attempts at alignment (take 2)

> "Wish I could find the first version"

- toc: false
- hidden: true
- branch: master
- categories: [riksdag, wav2vec, alignment]

In [4]:
TEST_A = "/Users/joregan/Playing/rd_ctm_edit/H5C120171011va"

In [5]:
lines = []
with open(TEST_A) as f:
    for line in f.readlines():
        lines.append(line.strip())

In [6]:
def accept_all(lines):
    outlines = []
    for line in lines:
        parts = line.split(" ")
        if parts[-1] == "cor":
            outlines.append(line)
        elif parts[-1] == "sub":
            parts[4] = parts[6]
            parts[-1] = "cor"
            outlines.append(" ".join(parts))
    return outlines

In [7]:
accept_all(lines)

['2442205210012872721 1 27.86 0.06 Herr 1.0 Herr cor',
 '2442205210012872721 1 28.0 0.48 talman! 1.0 talman! cor',
 '2442205210012872721 1 28.72 0.96 Riksdagsledamöter! 1.0 Riksdagsledamöter! cor',
 '2442205210012872721 1 30.16 0.82 Allianspartierna 1.0 Allianspartierna cor',
 '2442205210012872721 1 31.08 0.56 Moderaterna, 1.0 Moderaterna, cor',
 '2442205210012872721 1 32.08 0.74 Centerpartiet, 1.0 Centerpartiet, cor',
 '2442205210012872721 1 33.02 0.579 Liberalerna 1.0 Liberalerna cor',
 '2442205210012872721 1 33.74 0.079 och 1.0 och cor',
 '2442205210012872721 1 33.88 0.759 Kristdemokraterna 1.0 Kristdemokraterna cor',
 '2442205210012872721 1 34.76 0.399 föreslår 1.0 föreslår cor',
 '2442205210012872721 1 35.22 0.079 som 1.0 som cor',
 '2442205210012872721 1 35.36 0.579 riksdagens 1.0 riksdagens cor',
 '2442205210012872721 1 36.02 0.239 förste 1.0 förste cor',
 '2442205210012872721 1 36.32 0.179 vice 1.0 vice cor',
 '2442205210012872721 1 36.58 0.34 talman 1.0 talman cor',
 '24422052

In [8]:
def ctm_to_timed(lines):
    output = []
    for line in lines:
        parts = line.split(" ")
        start = float(parts[2])
        dur = float(parts[3])
        output.append({
            "start": start,
            "end": start + dur,
            "text": parts[6]
        })
    return output

In [9]:
side_a = ctm_to_timed(accept_all(lines))

In [10]:
phonfile = "/Users/joregan/Playing/rd_phonetic/2442205210012872721_480p.json"

In [11]:
import json
with open(phonfile) as f:
    pieces = json.load(f)

In [12]:
def hf_json_to_timed(data):
    output = []
    for chunk in data["chunks"]:
        output.append({
            "start": chunk["timestamp"][0],
            "end": chunk["timestamp"][1],
            "text": chunk["text"]
        })
    return output

In [13]:
side_b = hf_json_to_timed(pieces)

In [14]:
def prune_to_other(left, right, fudge=0.5):
    output = []
    for item in right:
        if item["start"] < left[0]["start"] - fudge:
            continue
        elif item["end"] > left[-1]["end"] + fudge:
            continue
        else:
            output.append(item)

    return left, output

In [15]:
new_a, new_b = prune_to_other(side_a, side_b)

In [16]:
def cost(a, b):
    starts = abs(a["start"] - b["start"])
    ends = abs(a["end"] - b["end"])
    return starts + ends

In [17]:
def in_start_range(a, b, range=0.2):
    return abs(a["start"] - b["start"]) <= range

def in_end_range(a, b, range=0.2):
    return abs(a["end"] - b["end"]) <= range

def in_range(a, b, range=0.2):
    r_start = in_start_range(a, b, range)
    r_end = in_end_range(a, b, range)
    return r_start or r_end


In [35]:
def falls_between(a1, a2, b):
    if b["end"] <= a2["start"] and b["start"] >= a1["end"]:
        return True
    return False

In [18]:
import numpy as np

In [34]:
def align_times(new_a, new_b):
    s1 = len(new_a)
    s2 = len(new_b)

    additionals = []

    dist_matrix = np.matrix(np.ones((s1, s2)))
    DEBUG = False

    for i in range(s1):
        for j in range(s2):
            if not in_range(new_a[i], new_b[j]):
                continue
            if i < s1 and falls_between(new_a[i], new_a[i + 1], new_b[j]):
                if DEBUG:
                    print(new_a[i], new_a[i + 1], new_b[j])
                additionals.append((new_a[i], new_a[i + 1], new_b[j]))
                pair_cost = 1.
            elif i == s1 and True:

                additionals.append((new_a[i], {}, new_b[j]))
            pair_cost = cost(new_a[i], new_b[j])
            dist_matrix[i, j] = pair_cost
            if DEBUG:
                print(new_a[i], new_b[j], pair_cost)
    return dist_matrix

In [ ]:
s1 = len(new_a)
s2 = len(new_b)

dist_matrix = np.matrix(np.ones((s1, s2)))
DEBUG = False

for i in range(s1):
    for j in range(s2):
        if not in_range(new_a[i], new_b[j]):
            continue
        pair_cost = cost(new_a[i], new_b[j])
        dist_matrix[i, j] = pair_cost
        if DEBUG:
            print(new_a[i], new_b[j], pair_cost)

In [31]:
dist_matrix = align_times(new_a, new_b)

In [32]:
import pandas as pd
df = pd.DataFrame(data=dist_matrix,index=[x["text"] for x in new_a], columns=[x["text"] for x in new_b])

In [33]:
df

,ɑː,tɑːlman,rɪksasleːda,møːtœ̞,<pa>,al,aspatiːæɳa,mʊdɑːtœ̞ɔɳa,<pa>,sentə,...,fœ̞ːesoː,sɔm,rɪksdɑːɡəns,fœ̞st,viːsə,tɑːlman,<pa>,eva,tareːn,fɪne
Herr,0.02,0.82,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
talman!,0.72,0.12,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
Riksdagsledamöter!,1.00,1.00,0.52,0.62,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
Allianspartierna,1.00,1.00,1.00,1.00,1.1,0.72,0.16,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
"Moderaterna,",1.00,1.00,1.00,1.00,1.0,1.00,1.00,0.02,0.88,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
"Centerpartiet,",1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,0.54,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
Liberalerna,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
och,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
Kristdemokraterna,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,1.000,1.000,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
föreslår,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,...,0.059,0.601,1.000,1.000,1.000,1.00,1.0,1.00,1.00,1.00
